**Demo for `teneva.collection.tensors`**

---

This module contains the collection of functions for explicit construction of
various useful TT-tensors (delta function, polynomial and others).

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `tensor_const`

Build TT-tensor (with TT-rank equals 1) with all values equal to given number.

In [2]:
n = [10] * 5                        # Shape of the tensor  
Y = teneva.tensor_const(n, v=42.)   # Tensor of all 42
teneva.show(Y)                      # Print the resulting TT-tensor
Y_full = teneva.full(Y)
print()
print(f'Min value : {np.min(Y_full)}')
print(f'Max value : {np.max(Y_full)}')

TT-tensor     5D : |10| |10| |10| |10| |10|
<rank>  =    1.0 :    \1/  \1/  \1/  \1/

Min value : 42.00000000000003
Max value : 42.00000000000003


We can, among other things, build the TT-tensor equal to zero everywhere:

In [3]:
n = [10] * 5                        # Shape of the tensor  
Y = teneva.tensor_const(n, v=0.)    # Tensor of all zeros
teneva.show(Y)                      # Print the resulting TT-tensor
Y_full = teneva.full(Y)
print()
print(f'Min value : {np.min(Y_full)}')
print(f'Max value : {np.max(Y_full)}')

TT-tensor     5D : |10| |10| |10| |10| |10|
<rank>  =    1.0 :    \1/  \1/  \1/  \1/

Min value : 0.0
Max value : 0.0


Note that the given value is distributed evenly over the TT-cores:

In [4]:
Y = teneva.tensor_const([10] * 100, v=42**100)
print(Y[5].reshape(-1))

[42. 42. 42. 42. 42. 42. 42. 42. 42. 42.]


We can also set multi-indices in which the tensor is forced to zero (note that this will also necessarily lead to the appearance of other zero elements):

In [5]:
n = [10] * 5                        # Shape of the tensor
I = [                               # Multi-indices for zeros
    [0, 0, 0, 0, 0],
    [1, 2, 3, 4, 5],
    [9, 9, 9, 9, 9],
]
Y = teneva.tensor_const(n, v=42., I_zero=I)

print(f'Y at I[0]           :', teneva.get(Y, I[0]))
print(f'Y at I[1]           :', teneva.get(Y, I[1]))
print(f'Y at I[2]           :', teneva.get(Y, I[2]))

Y_full = teneva.full(Y)

print(f'Num of zero items   :', np.sum(Y_full < 1.E-20))
print(f'Mean non zero value :', np.sum(Y_full) / np.sum(Y_full > 1.E-20))

Y at I[0]           : 0.0
Y at I[1]           : 0.0
Y at I[2]           : 0.0
Num of zero items   : 27100
Mean non zero value : 42.00000000000001


Then we specify multi-indices in which the tensor is forced to zero, we can also set one multi-index, which will not affected by zero multi-indices:

In [6]:
n = [10] * 5                        # Shape of the tensor
i = [5, 5, 5, 5, 5]                 # Multi-index for non-zero item
I = [                               # Multi-indices for zeros
    [0, 0, 0, 0, 0],
    [1, 2, 3, 4, 5],
    [9, 9, 9, 9, 9],
]
Y = teneva.tensor_const(n, v=42., i_non_zero=i, I_zero=I)

print(f'Y at i              :', teneva.get(Y, i))
print(f'Y at I[0]           :', teneva.get(Y, I[0]))
print(f'Y at I[1]           :', teneva.get(Y, I[1]))
print(f'Y at I[2]           :', teneva.get(Y, I[2]))

Y_full = teneva.full(Y)

print(f'Num of zero items   :', np.sum(Y_full < 1.E-20))
print(f'Mean non zero value :', np.sum(Y_full) / np.sum(Y_full > 1.E-20))

Y at i              : 42.00000000000003
Y at I[0]           : 0.0
Y at I[1]           : 0.0
Y at I[2]           : 0.0
Num of zero items   : 27100
Mean non zero value : 42.00000000000001


Note, if we set too many multi-indices in which the tensor is forced to zero (under which it will be impossible to keep a non-zero item), then it will lead to error:

In [7]:
n = [2] * 5                         # Shape of the tensor
i = [1, 1, 1, 1, 1]                 # Multi-index for non-zero item
I = teneva.sample_lhs(n, 100)       # Multi-indices for zeros

try:
    Y = teneva.tensor_const(n, v=42., i_non_zero=i, I_zero=I)
except ValueError as e:
    print('Error :', e)

Error : Can not set zero items


## Function `tensor_delta`

Build TT-tensor that is zero everywhere except for a given multi-index.

In [8]:
n = [20, 18, 16, 14, 12]            # Shape of the tensor
i = [ 1,  2,  3,  4,  5]            # The multi-index for nonzero element
v = 42.                             # The value of the tensor at multi-index "i"
Y = teneva.tensor_delta(n, i, v)    # Build TT-tensor

teneva.show(Y)

TT-tensor     5D : |20| |18| |16| |14| |12|
<rank>  =    1.0 :    \1/  \1/  \1/  \1/


Let check the result:

In [9]:
Y_full = teneva.full(Y)             # Transform TT-tensor to full format
i_max = np.argmax(Y_full)           # Find multi-index and value for max
i_max = np.unravel_index(i_max, n)
y_max = Y_full[i_max]

# Find number of nonzero tensor items:
s = len([y for y in Y_full.flatten() if abs(y) > 1.E-10])
    
print(f'The max value multi-index:', i_max)
print(f'The max value            :', y_max)
print(f'Number of nonzero items  :', s)

The max value multi-index: (1, 2, 3, 4, 5)
The max value            : 42.00000000000003
Number of nonzero items  : 1


We can also build some multidimensional TT-tensor by "delta" function and check the norm of the result:

In [10]:
d = 100                             # Dimension of the tensor
n = [20] * d                        # Shape of the tensor
i = [3] * d                         # The multi-index for nonzero element
v = 42.                             # The value of the tensor at multi-index "k"
Y = teneva.tensor_delta(n, i, v)    # Build TT-tensor

teneva.norm(Y)

42.00000000000021

## Function `tensor_poly`

Build TT-tensor that is polynomial like scale * (index + shift)^power.

In [11]:
n = [10] * 5                        # Shape of the tensor
shift = np.array([2, 3, 2, 3, 2])   # Shift value
scale = 5.                          # Scale
power = 3                           # Power
Y = teneva.tensor_poly(n, shift, power, scale)
teneva.show(Y)                      # Print the resulting TT-tensor

TT-tensor     5D : |10| |10| |10| |10| |10|
<rank>  =    2.0 :    \2/  \2/  \2/  \2/


We can check the result:

In [12]:
i = [2, 3, 3, 4, 5]

y_appr = teneva.get(Y, i)
y_real = scale * np.sum((i + shift)**power)

print(y_appr)
print(y_real)

5455.0
5455.0


In [13]:
i = np.zeros(5)

y_appr = teneva.get(Y, i)
y_real = scale * np.sum((i + shift)**power)

print(y_appr)
print(y_real)

390.0
390.0


The value of "shift" may be also scalar:

In [14]:
Y = teneva.tensor_poly(n, 42., power, scale)
teneva.show(Y)

TT-tensor     5D : |10| |10| |10| |10| |10|
<rank>  =    2.0 :    \2/  \2/  \2/  \2/


## Function `tensor_rand`

Construct random TT-tensor.

In [15]:
n = [12, 13, 14, 15, 16]      # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]        # TT-ranks for TT-tensor
Y = teneva.tensor_rand(n, r)  # Build random TT-tensor
teneva.show(Y)                # Print the resulting TT-tensor

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    3.6 :    \2/  \3/  \4/  \5/


If all inner TT-ranks are equal, we may pass it as a number:

In [16]:
n = [12, 13, 14, 15, 16]      # Shape of the tensor
r = 5                         # TT-ranks for TT-tensor
Y = teneva.tensor_rand(n, r)  # Build random TT-tensor
teneva.show(Y)                # Print the resulting TT-tensor

TT-tensor     5D : |12| |13| |14| |15| |16|
<rank>  =    5.0 :    \5/  \5/  \5/  \5/


---